<a href="https://colab.research.google.com/github/VitorFRodrigues/Polo_Tech_Americanas/blob/main/Polo_Tech/Modulo_05/Projeto/Projeto_Mod05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
url = 'https://raw.githubusercontent.com/VitorFRodrigues/Polo_Tech_Americanas/main/Polo_Tech/Modulo_05/Projeto/dataset/fipe_2022.csv'

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# importe as principais bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDis

In [8]:
dados = pd.read_csv(url)
dados

,year_of_reference,month_of_reference,fipe_code,authentication,brand,model,fuel,gear,engine_size,year_model,avg_price_brl,age_years
0,2022,January,038001-6,vwmrywl5qs,Acura,NSX 3.0,Gasoline,manual,3.0,1995,43779.0,28
1,2022,January,038001-6,t9mt723qhz,Acura,NSX 3.0,Gasoline,manual,3.0,1994,42244.0,29
2,2022,January,038001-6,tr5wv4z21g,Acura,NSX 3.0,Gasoline,manual,3.0,1993,40841.0,30
3,2022,January,038001-6,s2xxsjz3mt,Acura,NSX 3.0,Gasoline,manual,3.0,1992,39028.0,31
4,2022,January,038001-6,rtm9gj7zk8,Acura,NSX 3.0,Gasoline,manual,3.0,1991,35678.0,32
...,...,...,...,...,...,...,...,...,...,...,...,...
290270,2022,December,061001-1,lzrp3rpklh8q,Walk,Buggy Walk Sport 1.6 8V 58cv,Gasoline,manual,1.6,2009,22894.0,14
290271,2022,December,061001-1,lrfw427w9v5h,Walk,Buggy Walk Sport 1.6 8V 58cv,Gasoline,manual,1.6,2008,22270.0,15
290272,2022,December,061001-1,lh98rfxnhy18,Walk,Buggy Walk Sport 1.6 8V 58cv,Gasoline,manual,1.6,2007,21663.0,16
290273,2022,December,061001-1,k9dtwsps5sy1,Walk,Buggy Walk Sport 1.6 8V 58cv,Gasoline,manual,1.6,2006,21073.0,17


In [9]:
dados.isna().sum()

year_of_reference     0
month_of_reference    0
fipe_code             0
authentication        0
brand                 0
model                 0
fuel                  0
gear                  0
engine_size           0
year_model            0
avg_price_brl         0
age_years             0
dtype: int64

In [15]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290275 entries, 0 to 290274
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   year_of_reference   290275 non-null  int64  
 1   month_of_reference  290275 non-null  object 
 2   fipe_code           290275 non-null  object 
 3   authentication      290275 non-null  object 
 4   brand               290275 non-null  object 
 5   model               290275 non-null  object 
 6   fuel                290275 non-null  object 
 7   gear                290275 non-null  object 
 8   engine_size         290275 non-null  float64
 9   year_model          290275 non-null  int64  
 10  avg_price_brl       290275 non-null  float64
 11  age_years           290275 non-null  int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 26.6+ MB


In [14]:
for i in dados.columns.tolist():
  print(f'{i}: {dados[i].unique()}')
  print('')

year_of_reference: [2022]

month_of_reference: ['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']

fipe_code: ['038001-6' '038002-4' '038003-2' ... '005534-4' '005535-2' '005536-0']

authentication: ['vwmrywl5qs' 't9mt723qhz' 'tr5wv4z21g' ... 'lh98rfxnhy18' 'k9dtwsps5sy1'
 'j7gt9020nqvt']

brand: ['Acura' 'Agrale' 'Alfa Romeo' 'AM Gen' 'Asia Motors' 'ASTON MARTIN'
 'Audi' 'Baby' 'BMW' 'BRM' 'CAB Motors' 'Cadillac' 'CBT Jipe' 'CHANA'
 'CHANGAN' 'Caoa Chery' 'GM - Chevrolet' 'Chrysler' 'Citroën'
 'Cross Lander' 'Daewoo' 'Daihatsu' 'Dodge' 'EFFA' 'Engesa' 'Envemo'
 'Ferrari' 'Fiat' 'Fibravan' 'Ford' 'FOTON' 'Fyber' 'GEELY' 'GREAT WALL'
 'HAFEI' 'Honda' 'Hyundai' 'Isuzu' 'IVECO' 'JAC' 'Jaguar' 'Jeep' 'JINBEI'
 'JPX' 'Kia Motors' 'Lada' 'Land Rover' 'Lexus' 'LIFAN' 'LOBINI' 'Lotus'
 'Mahindra' 'Maserati' 'Matra' 'Mazda' 'Mclaren' 'Mercedes-Benz' 'Mercury'
 'MG' 'MINI' 'Mitsubishi' 'Nissan' 'Peugeot' 'Plymouth' 'Pontiac'
 'Porsche

In [20]:
dados.brand.value_counts().head(50)

GM - Chevrolet     28179
VW - VolksWagen    28142
Fiat               26563
Ford               24150
Mercedes-Benz      17470
Renault            15194
Audi               12168
Peugeot            11132
Toyota             10309
Mitsubishi         10182
BMW                10060
Citroën             8780
Hyundai             7485
Land Rover          6973
Nissan              6885
Porsche             6800
Volvo               6069
Honda               5710
Kia Motors          5473
Subaru              4138
Suzuki              3867
Jeep                3656
Jaguar              2788
MINI                2352
Chrysler            1824
Lexus               1749
Caoa Chery          1583
Maserati            1367
Dodge               1206
SSANGYONG           1174
JAC                 1103
IVECO                991
Agrale               960
Fibravan             921
Ferrari              852
EFFA                 820
Troller              765
BRM                  630
Baby                 580
Alfa Romeo           504


In [46]:
dados.query("brand == 'Ferrari'")['model'].value_counts().count()

23

In [ ]:
for 

## Roteiro de itens 

- plot preços versus tipo de comb
- plot preços versus tipo de marcha
- plot preços versus tamanho
- random undersampling
- Utilizar Marca JAC para fazer o modelo
- Dropar 
